#####***Global variables and config settings***

In [6]:
import pandas as pd
import numpy as np
import datetime as dt
import db_connections as db
import os

# input_file = 'dtms_mem_final_13May_thru_7Jun.xlsx'
# input_path = 'N:\\N Desktop\\Pandas xdock\\'


new_xdock_path = 'N:\\N Desktop\\Pandas xdock\\'
new_xdock_file = 'future_xdock.xlsx'


exists = os.path.isfile('%s%s' % (new_xdock_path,new_xdock_file)) 
if not exists:
    createnewx = pd.DataFrame(columns=('store','start_date','new_receipt','new_isd'))
    createnewx.to_excel('%s%s' % (new_xdock_path,new_xdock_file), sheet_name='future', index=False, encoding='utf-8')



output_file = 'xdock_output.csv'
output_errors = 'xdock_errors.csv'
output_multiple_counts = 'multiple_counts.csv'
output_path = 'N:\\N Desktop\\Pandas xdock\\'


# good practice is to use a start of Friday
# and an end of Monday to capture errors that cut over weekends
ship_start = '2020-10-23'
ship_end = '2020-12-18'


# print "Importing %s from %s" % (input_file,input_path)

# print "Saving parsed ODS file to: %s" % output_file


In [7]:
# This is a flat file to input future xdock changes
# Xdock codes cannot be udpdated ahead of time
# They must be manually adjusted if referencing future ODS dates

future_xdock = pd.read_excel('%s%s'% (new_xdock_path,new_xdock_file), sheet_name='future')
if future_xdock.empty:
    print 'empty'

# fill in nulls with default text
future_xdock['new_receipt'] = future_xdock['new_receipt'].fillna('blank')
future_xdock['new_isd'] = future_xdock['new_isd'].fillna('blank')
    
# Strip out whitespace
future_xdock['new_receipt'] = future_xdock['new_receipt'].apply(lambda x:x.strip())
future_xdock['new_isd'] = future_xdock['new_isd'].apply(lambda x:x.strip())

# Convert to uppercase
future_xdock['new_receipt'] = future_xdock['new_receipt'].apply(lambda x:x.upper())
future_xdock['new_isd'] = future_xdock['new_isd'].apply(lambda x:x.upper())

# Convert store number to INT - it will be used in a join
future_xdock['store'] = future_xdock['store'].astype(np.int64)
# Convert start-date to datetime - this is mostly for when the file is empty, to get it in the right datatype
future_xdock['start_date'] = future_xdock['start_date'].astype('datetime64[ns]')


future_xdock.head()

,store,start_date,new_receipt,new_isd
0,7363,2020-10-19,S,MT
1,10397,2020-10-19,R,FS


#####***Import the current ODS from the Oracle DB***

DF Name: ***ods***

In [8]:
# ods = pd.read_excel('%s%s' % (input_path,input_file), sheet_name='ods')

print "Extracting ship-dates between %s and %s" %(ship_start,ship_end)

ods = pd.read_sql("""select
xsods.STORE_NUMBER "STORE_NUMBER",
xsods.CUSTOMER_NUMBER "CUSTOMER_NUMBER",
xsods.CUSTOMER_SITE_NUMBER "CUSTOMER_SITE_NUMBER",

xdpm.DP_NAME "DP_NAME",

to_date(xsods.SHIP_DATE,'DD-MON-YY') "SHIP_DATE",
xsods.DELIVERY_DATE "DELIVERY_DATE",

--nvl(xdps.X_DOCK_LOCATION,'CDC1999') "X_DOCK_LOCATION",
--nvl(xdps.RECEIPT_DAY_CDC,'X') "RECEIPT_DAY_CDC"

xdps.X_DOCK_LOCATION "X_DOCK_LOCATION",
xdps.RECEIPT_DAY_CDC "RECEIPT_DAY_CDC"

from apps.XXOM_ODS_SCHED_SUM xsods
join apps.XXOM_SOURCING_ODS_CTRL xsctrl on xsctrl.STATUS='active' and xsods.CONC_REQUEST=xsctrl.CONC_REQUEST
join apps.XXOM_DSTRB_POINT_MSTR_V xdpm on xsods.DP_NUMBER=xdpm.DSTRB_POINT_NUMBER
join apps.XXOM_DSTRB_POINT_STORES_V xdps on xsods.CUSTOMER_SITE_NUMBER=xdps.PARTY_SITE_NUMBER and xdpm.DSTRB_POINT_ID=xdps.DSTRB_POINT_ID


where 1=1
and xdpm.DSTRB_POINT_TYPE_CODE = 'RDC'
and xsods.SCHEDULE_TYPE in ('Standard','Alternate')
and xdps.X_DOCK_LOCATION is not null
--and (xdps.X_DOCK_LOCATION is not null or xsods.STORE_NUMBER in (286,298,2203,2206,2208,2212,2238,2322,2360,2374,2377,2383,2384,2385,2387,2393,2407,2408,2420,2429,2433,2437,2447,2458,2459,2464,2466,2472,2478,2487,2497,2520,2526,2538,2541,2544,2551,2561,2562,2575,2581,2588,2589,2608,2633,2640,2656,2659,2663,2664,2670,2673,2687,2691,2699,2701,2703,2723,2724,2725,2727,2738,2745,2758,2759,2760,2763,2790,2828,2856,7931,7988,7990,7991,8123,8124,8690,8691,8692,8693,8733,8734,8871,8920,8921,8955,9355,9398,9399,9431,9434,9520,9560,9561,9562,9571,9759,9877,9880,9890,9891,9951,9952,9982,9983,10029,10073,10075,10154,10318,10373,10374,10454,10509,10516,10559,10595,10637,10638,10639,10689,10735,10736,10737,10787,10788,10827,10829,10974,11180,11181,11281,11501,11502,11503,11505,11506,11509,11510,11652,11696,11739,11794,11796,11798,11845,13249,13343,13344,13345,13376,13443,13445,13471,13618,13755,13949,13950,13952,13961,13973,13974,13975,14259,16953,19031,19091,19563,19868,20250,21137,21212,21213,21375,21753,23154,23159,23163,23620,24257,24568,24569,24662,27232,27458,27624,27627,28174,28175,28851,29090,29202,29428,29444,29458,47094,47095,47096,47097,48199,48250,48415,48416,48567,48568,48569,48571,48572,48636,50688,50719,50837,51519,51647,51734,52318,52319,52454,53792,53846,54203,54204,55613,55718,54447,55584,56306,57269,57813,59055,57671,58693,60569))

and to_date(xsods.SHIP_DATE,'DD-MON-YY') between to_date('%s','YYYY-MM-DD') and to_date('%s','YYYY-MM-DD')
""" % (ship_start,ship_end),db.sbp609)

ods.head()

# ods.to_csv('%s%s' % (output_path,output_file),index=False, encoding='utf-8')

# ods.dtypes

Extracting ship-dates between 2020-10-23 and 2020-12-18


,store_number,customer_number,customer_site_number,dp_name,ship_date,delivery_date,x_dock_location,receipt_day_cdc
0,75413,4114870,234997,RDC-Nashville,2020-10-26,2020-10-29,CDC1013,TS
1,75413,4114870,234997,RDC-Nashville,2020-10-29,2020-11-02,CDC1013,TS
2,75413,4114870,234997,RDC-Nashville,2020-11-02,2020-11-05,CDC1013,TS
3,75413,4114870,234997,RDC-Nashville,2020-11-05,2020-11-09,CDC1013,TS
4,75413,4114870,234997,RDC-Nashville,2020-11-09,2020-11-12,CDC1013,TS


#####***Import Xdock Codes for future changes***

DF Name: ***future_xdock***

In [9]:

# value maps to help convert between days and dates
# and the delta between dates

day_to_num = {'Sunday':-3,'Monday':-2,'Tuesday':-1,'Wednesday':0, 'Thursday':1,\
           'Friday':2,'Saturday':3}

code_to_num = {'U':-3,'M':-2,'T':-1,'W':0, 'R':1,'F':2,'S':3,'X':9}
num_to_code = {-3:'U', -2:'M', -1:'T', 0:'W', 1:'R',2:'F',3:'S'}

days = ['M','T','W','R','F','S','U']

exempt_codes = ['REMOVE']


### Update the xdock code for future xdock changes
# from flat-file - does not get updated automatically

def update_xdock(currcode,deldate, startdate,newcode):
    if deldate >= startdate:
        return newcode
    else:
        return currcode
### end of update_xdock



### parse the xdock string and return a single value for the CDC receipt "day"
# Correct "day" is within range between ship and delivery
# and if multiple, choose the one closer to delivery
def parse_xdock (s,code,d,itlt,xcdc):
   
    # hold valid xdock receipt days in this string
    valid = str()
    
    # this will be used to find correct receipt for delivery-date 
    # CDCs with lots of remote stores may have more lead-time between
    # receipt and delivery
    rec_lt = 1
    if xcdc == 'CDC1000' or xcdc == 'CDC1013':
        rec_lt = 2
    
    # if ship is >= 7 from receipt, then all days will be valid by definition
    # so just proceed to find closest receipt to delivery
    if itlt >=7:
        valid = code

    # If a store is being removed from xdock, we want to exclude their new ODS from xdock valiation
    # since they will no longer have an xdock code, I'm using "REMOVE" instead
    # Instead of hard-coding, i'm using a list instead - in case I want to add new codes in the future
    # this says, just ignore the special code as return it as-is.
    if code in exempt_codes:
        return code
    
    # iterate over each character in the string and
    # evaluate if that day is between ship and delivery days
    if itlt < 7:
        for i,c in enumerate(code):
            
            
            # check for non-valid day codes - if found, just return "X" which will denote an error
            if c not in days:
                return "X"
            
            # This is for a special case where order is shipped and received on a weekend
            # but ship-date cannot be weekend date due to Oracle calendar closures
            # back up the ship-range to include Sat and Sun
            if s == -2 and c in ('SU') and itlt <= 1:
                s = 3               
            
            #convert the current character to a number in our value map
            x = code_to_num[c]
            # make a copy of the delivery day so that we can edit it under different conditions
            dd = d
            # adding 7 will help wrap around the array when the receipt is the following week
            if s > x:
                x = x + 7
            # same "wrap" logic for delivery in following week
            if s > d:
                dd = d + 7

            #check if the current xdock day is between ship and delivery and save it
            if s <= x < dd:
                valid += c

    # if there is only one valid value after parsing string, return it
    if len(valid) == 1:
        return valid

    # the next block starts with the day before delivery day (receipt and delivery can't be same)
    # modified for remote CDCs with 2-day between receipt and delivery
    # and checks to see if it is a valid day and return it if it is valid
    # otherwise, try day before that, etc...
    
    # start with day(s) before delivery as defined by the receipt lead-time in rec_lt
    index = days.index(num_to_code[d])-rec_lt
    if index < 0:
        index = 7 + index

    # it can only be a weekday, so only loop over 6 days, use counter for loop condition
    counter = 0
    while counter < 6:
        # this is to wrap around the array once we get out of range
        # we work back from delivery to find the next valid day
        if index < 0:
            index = 6
        # check if day is in our valid list    
        if days[index] in valid:
            return days[index]
        index = index - 1
        counter = counter + 1  

    # return X if you cannot find a valid receipt
    # this will identify the record as an error
    return "X"
### End of parse_xdock



### Calculate the CDC receipt-date
# everything has already been converted into numeric values
# just check if it wraps to the previous week
# and count back from the delivery date
def receipt_date(x,d,ddate):
    if x == 9:
        return np.nan
    if x > d:
        x = x - 7
    delta = d - x
    return ddate - dt.timedelta(days=delta)
### end of receipt_date

# Convert the store number to INT and then merge with the future changes
ods['store_number'] = ods['store_number'].astype(np.int64)
ods = pd.merge(ods, future_xdock, left_on='store_number', right_on='store', how='left')

# Update the CDC receipt code in case of a future schedule change
ods['receipt_day_cdc'] = ods.apply(lambda x: update_xdock(x['receipt_day_cdc'],x['delivery_date'],x['start_date'],x['new_receipt']), axis=1)

# Delete the ODS for when store is moving from xdock to LTL
# Instead of a xdock code, use "REMOVE" in the "future xdock" file for both Receipt and ISD
ods = ods[(ods['receipt_day_cdc'] != 'REMOVE')]




# convert date to weekday name
ods['ship_num'] = ods['ship_date'].dt.weekday_name
ods['del_num'] = ods['delivery_date'].dt.weekday_name

# apply the value map to the weekday name 
# to get the weekday number in range -3 to 3 (Sun thru Sat)
ods['ship_num'] = ods['ship_num'].map(day_to_num)
ods['del_num'] = ods['del_num'].map(day_to_num)

# calculate the In-Transit-Lead_Time (itlt) 
# values of 0 and >=7 require special logic
ods['itlt'] = (ods['delivery_date'] - ods['ship_date']).dt.days


# create a column with the correct single-character xdock code
# the function will parse through multi-delivery strings
# it will also validate if a single code is within the correct range
# a single day-code (i.e. M,T,W,R,F,S,U) will be returned
# or, an X will be returned when a valid value cannot be found
ods['single_xdock'] = ods.apply(lambda x: parse_xdock(x['ship_num'],x['receipt_day_cdc'],x['del_num'],x['itlt'],x['x_dock_location']),axis=1)

# apply the value map to the single-character xdock code
ods['xdock_num'] = ods['single_xdock'].map(code_to_num)

# calculate the CDC Receipt Date using the value map numbers
ods['cdc_recpt_date'] = ods.apply(lambda x: receipt_date(x['xdock_num'],x['del_num'],x['delivery_date']), axis=1)

# export the entire ODS to a file
ods.to_csv('%s%s' % (output_path,output_file),index=False, encoding='utf-8')

# Identify the lines with errors i.e. the program was not able to pick a validate receipt date
# based on the available xdock codes, and ship and delivery dates
# 9 was just a placeholder in a int field
xdock_errors = ods[(ods['xdock_num'] == 9)]

# Export the error message and error file, but only if errors exist
print "Saving parsed ODS file to: %s%s" % (output_path,output_file)
if xdock_errors.empty:
    pass
else:
    xdock_errors.to_csv('%s%s' % (output_path,output_errors),index=False, encoding='utf-8')
    print "***Errors were generated, saving file to: %s%s***" % (output_path,output_errors)


Saving parsed ODS file to: N:\N Desktop\Pandas xdock\xdock_output.csv


In [10]:

# The next steps check for when there are 2 different ship-dates for the same cdc receipt-date.
# This is a bad scenario because xdock is based on an SOP and generally all stores
# with the same CDC and receipt day should drop on the same day
# There are known exceptions of course

# These are the unique fields, copy them into their own datafrome
flatten_ods = ods[['ship_date','cdc_recpt_date','x_dock_location','single_xdock']].copy()

# Remove duplicates, i.e. basically, all of the the specfic store-info has been stripped out
# and this just leaves the basic schedule
flatten_ods.drop_duplicates(inplace=True)

# Use groupby to apply an aggragate fuction, i.e. count the number of ship-dates for each CDC/Receipt combo
# Ideally each line should be unique because we removed duplicates
bad_count = flatten_ods.groupby(['x_dock_location','single_xdock','cdc_recpt_date'])['ship_date'].count().reset_index(name='count')

# The count is the number of unique ship-dates for each receipt-date
# They should be uniform, so anything with more than one is bad
bad_count = bad_count[bad_count['count'] > 1].copy()

# Save the offending records
bad_count.to_csv('%s%s' % (output_path,output_multiple_counts),index=False, encoding='utf-8')


